This code is based on https://huggingface.co/stabilityai/stable-diffusion-x4-upscaler

In [2]:
import requests
from PIL import Image
from io import BytesIO
from diffusers import StableDiffusionUpscalePipeline
import torch
import cv2
import numpy as np

/Users/idilunlu/miniconda3/envs/upscaler/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
model_id = "stabilityai/stable-diffusion-x4-upscaler"
pipeline = StableDiffusionUpscalePipeline.from_pretrained(model_id, torch_dtype=torch.float16)
pipeline = pipeline.to("mps")

Loading pipeline components...: 100%|██████████| 6/6 [00:05<00:00,  1.17it/s]


In [4]:
url = "https://huggingface.co/datasets/hf-internal-testing/diffusers-images/resolve/main/sd2-upscale/low_res_cat.png"
response = requests.get(url)
low_res_img = Image.open(BytesIO(response.content)).convert("RGB")
low_res_img = low_res_img.resize((128, 128))

In [5]:
image_path = "0_10.png"  # Change this to the path of your image file
low_res_img = Image.open(image_path).convert("RGB")
width, height = low_res_img.size
print("Original image size:", width, "x", height)
#low_res_img = low_res_img.resize((128, 128))

Original image size: 224 x 224


In [6]:
prompt = "high resolution image of a histopathology slide"
# this model doesn't work without text prompt 
# and the quality of generated image is highly dependent on the prompt
upscaled_image = pipeline(prompt=prompt, image=low_res_img).images[0]
upscaled_image.save("0_10_upsampled.png")

100%|██████████| 75/75 [08:58<00:00,  7.18s/it]


In [45]:
print(low_res_img)
print(upscaled_image.size)

<PIL.Image.Image image mode=RGB size=224x224 at 0x16B7123A0>
(896, 896)


### Next Steps
Might actually make more sense to crop first and then upscale the image from the computational perspective

In [55]:
def zoom_in(image_path, output_path):
    # Load the image
    img = cv2.imread(image_path)
    if img is None:
        print("Error: Image not found")
        return None

    height, width = img.shape[:2]

    # Calculate the coordinates of the new cropped region
    new_width, new_height = width // 2, height // 2
    x1, y1 = width // 4, height // 4  # Top left corner 
    x2, y2 = x1 + new_width, y1 + new_height  # Bottom right corner

    # Crop the image
    cropped_img = img[y1:y2, x1:x2]
    print("Cropped image size:", new_width, "x", new_height)

    # # Resize the cropped image to half of the original image's resolution
    # resized_img = cv2.resize(cropped_img, (width // 4, height // 4), 
    # interpolation=cv2.INTER_LINEAR)
    # print("Resized image size:", resized_img.shape[1], "x", resized_img.shape[0])

    # return resized_img
    cv2.imwrite(output_path, cropped_img)
    return cropped_img

In [1]:
image_path = '0_10_upsampled.png'
out_image_path = '0_20_zoomed.png'
out_image_path2 = '0_40_zoomed.png'
zoomed_image_20x = zoom_in(image_path, out_image_path)
zoomed_image_40x = zoom_in(out_image_path, out_image_path2 )

NameError: name 'zoom_in' is not defined

In [59]:
image_path = "0_20.png"  
ground_truth20x = Image.open(image_path).convert("RGB")
width, height = ground_truth20x.size
print("Original image size:", width, "x", height)
#ground_truth20x = ground_truth20x.resize((128, 128))

Original image size: 224 x 224
